In [1]:
from MetaLearner import *

In [2]:
import pickle
import numpy as np
import time

In [ ]:
# todo: https://stackoverflow.com/questions/24168569/how-to-add-names-to-a-numpy-array-without-changing-its-dimension

In [ ]:
# This file runs the experiment

In [3]:
tf.keras.backend.set_floatx('float64')

In [4]:
tf.keras.utils.set_random_seed(8953)

In [5]:
file_name = "/Users/arberimbibaj/Documents/Master Thesis ETH/DataSets /Generated/data_synthetic_one_run.pkl"
open_file = open(file_name, "rb")
data = pickle.load(open_file)
open_file.close()

In [6]:
n_setups = 1
sample_sizes = [500, 1000, 2000, 5000]
n_runs = 1

In [ ]:
learners = [TLearner('rf'), SLearner('rf'), XLearner('rf'), RLearner('rf'), DRLearner('rf'), RALearner('rf'),
            PWLearner('rf'), ULearner('rf'),
            TLearner('lasso'), SLearner('lasso'), XLearner('lasso'), RLearner('lasso'), DRLearner('lasso'),
            RALearner('lasso'), PWLearner('lasso'), ULearner('lasso'),
            TLearner('nn'), SLearner('nn'), XLearner('nn'), RLearner('nn'), DRLearner('nn'), RALearner('nn'),
            PWLearner('nn'), ULearner('nn')]

In [7]:
learners = [TLearner('nn'), SLearner('nn'), XLearner('nn'), RLearner('nn'), DRLearner('nn'), RALearner('nn'),
            PWLearner('nn'), ULearner('nn')]

In [ ]:
learners = [RLearner('nn')]

In [ ]:
# What we want: For each setting and base learner --> matrix which includes the MSE, columns: Metalearners + SampleSize, rows: Run(1-10)
# --> So we can plot the mean and 1se of the MSE for each Metalearner, each samplesize; giving the setting and baselearner
# --> number of plots: settings*baselearner.

In [8]:
results = []

for i in range(n_setups):
    results.append([])

for i in range(n_setups):
    for baselearner in range(1):
        results[i].append([])

results

[[[]]]

In [9]:
dim = data[0][0][0][0].shape[1]  # dimension of x


# helper function to get y, x, w, tau, out of data[][][][]
def get_variables(dataset, setup, samplesize, run, train_test):
    y = dataset[setup][samplesize][run][train_test][:, 0]
    x = dataset[setup][samplesize][run][train_test][:, 1:(dim - 2)]
    w = dataset[setup][samplesize][run][train_test][:, (dim - 2)]
    tau = dataset[setup][samplesize][run][train_test][:, (dim - 1)]
    return y, x, w, tau

In [10]:
b = 0
m = 0
s = 0

for i in range(n_setups):
    print(f'Setup: {i + 1}')
    # array for all mses for one setup
    setup_mse = np.empty(shape=(0, 24))
    s = 0  # restart index for samplesize
    for size in sample_sizes:
        print(f'Sample Size: {s + 1}')
        # array for all mses for one setup and samplesize.
        size_mse = np.empty(shape=(0, 24))
        for r in range(n_runs):
            print(f'Run: {r + 1}')
            # array for all mses in one setup, samplesize and run.
            mses = np.empty(shape=(1, 24))
            # get data for specific setup, samplesize and run.
            temp_y_train, temp_x_train, temp_w_train, temp_tau_train = get_variables(dataset=data, setup=i,
                                                                                     samplesize=s, run=r,
                                                                                     train_test=0)
            temp_y_test, temp_x_test, temp_w_test, temp_tau_test = get_variables(dataset=data, setup=i,
                                                                                 samplesize=s, run=r,
                                                                                 train_test=1)
            # restart index for metalearner
            m = 0
            for l in learners:
                if m % 8 == 0: print(f'BaseLearner')
                print(f'Learner {m + 1}: {l}')
                tic = time.time()
                # training and testing MetaLearner.
                learner = l
                learner.fit(temp_x_train, temp_y_train, temp_w_train)
                predictions = learner.predict(temp_x_test)
                temp_mse = ((predictions - temp_tau_test) ** 2).mean()
                # append mse of specific metalearner to 'mses'.
                mses[0, m] = temp_mse
                # print time
                toc = time.time()
                print(f'Time: {round(toc - tic, 4)} seconds.')
                # update index
                m += 1
            # append 'mses' to 'size_mse'.
            size_mse = np.append(size_mse, mses, axis=0)
        # append 'size_mse' to 'setup_mse'.
        setup_mse = np.append(setup_mse, size_mse, axis=0)
        # update index
        s += 1
    # append to results
    results[i][0] = setup_mse[:, 0:8]  # random forest
    #results[i][1] = setup_mse[:, 8:16]  # lasso
    #results[i][2] = setup_mse[:, 16:24]  # neural network





# this should make sense i guess! :)
# TODO: CHECK IF IT MAKES IT FASTER IF EVERYTHING IS EITHER IN NP OR IN LIST/ARRAY BASE PYTHON.
# TODO: DONE :)
# TODO: DONE :)
# TODO: DONE :)

### IT TOOK 6M 40S FOR --> SAMPLE SIZES [200, 200], TEST SIZE = 50, 2 SETUPS, 2 RUNS LOL

### 35min for all Baselearners, 1 Setup, 1 Run, 4 Sample Sizes  ()

Setup: 1
Sample Size: 1
Run: 1
BaseLearner
Learner 1: <MetaLearner.TLearner object at 0x288ca54e0>
Time: 3.8948 seconds.
Learner 2: <MetaLearner.SLearner object at 0x288cf3b80>
Time: 1.7718 seconds.
Learner 3: <MetaLearner.XLearner object at 0x288d73b20>
Time: 7.1268 seconds.
Learner 4: <MetaLearner.RLearner object at 0x288d86d70>
Time: 4.6949 seconds.
Learner 5: <MetaLearner.DRLearner object at 0x28909f2e0>
Time: 47.815 seconds.
Learner 6: <MetaLearner.RALearner object at 0x28912d9f0>
Time: 6.1021 seconds.
Learner 7: <MetaLearner.PWLearner object at 0x28919d3c0>
Time: 3.2073 seconds.
Learner 8: <MetaLearner.ULearner object at 0x2891d39d0>
Time: 4.9505 seconds.
Sample Size: 2
Run: 1
BaseLearner
Learner 1: <MetaLearner.TLearner object at 0x288ca54e0>
Time: 5.6017 seconds.
Learner 2: <MetaLearner.SLearner object at 0x288cf3b80>
Time: 3.4026 seconds.
Learner 3: <MetaLearner.XLearner object at 0x288d73b20>
Time: 10.8922 seconds.
Learner 4: <MetaLearner.RLearner object at 0x288d86d70>
Time:

In [ ]:
### Training tau0 and tau1 takes extremely long!! compared to mu0 and mu1 in lasso!!! probably because of the max_iter

In [ ]:
setup_mse

In [ ]:
# set n_trees to 1000 to gain time or maybe even 500

In [11]:
results[0][0]

array([[8.82242677e-01, 1.34512945e-01, 4.13139923e-01, 3.48320041e-01,
        1.30060889e-01, 5.51353617e-01, 6.02475083e+00, 1.04965039e+00],
       [8.21603347e-01, 2.02012140e-01, 4.50496680e-01, 1.74621963e+00,
        1.04222708e+01, 7.24155584e-01, 2.98105190e+01, 9.95188563e+00],
       [6.90702094e-01, 1.56491740e-01, 4.91725492e-01, 1.80013899e+00,
        2.46925936e+00, 6.09572286e-01, 3.56520118e+01, 1.90493953e+00],
       [3.35695025e-01, 4.81432903e-02, 2.13545315e-01, 7.92501269e-01,
        6.45091965e+00, 3.46110284e-01, 9.55532026e+01, 1.94123052e+00]])

In [ ]:
# 0.885576,0.134742,0.411388,0.344136,0.128143,0.588750,5.880639,1.026057
# 0.815042,0.201147,0.439917,1.577994,10.549566,0.710207,29.923138,10.357554
# 0.679716,0.154199,0.399313,1.942728,0.317815,0.607700,37.089789,1.663813
# 0.285524,0.053260,0.222201,0.769516,6.082601,0.306323,92.644239,2.648658



In [ ]:
# results[0][2]
# 0.694569,0.167760,0.567150,2.997832,0.467543,0.944197,16.717111,12.709246
# 0.610678,0.187881,0.759497,22.874956,6.021213,1.137736,58.716848,1734.143168
# 0.864565,0.183118,0.793892,7.919859,10.800849,1.035885,72.427161,343.769983
# 0.233476,0.074466,0.366872,3.135558,34.819663,0.473895,124.064119,1850.331233

# --> PW and U-learner are bad here (why?)
# --> looks nice otherwise :)

In [ ]:
# Results[0][1]
# 2.308568,1.004442,1.526483,0.028155,2.156328,2.115060,0.036090,0.033795
# 1.791405,1.193947,1.260019,0.044737,0.323602,1.588854,5.308641,0.042311
# 0.315428,0.336822,0.348231,0.023158,0.071988,0.244252,0.000031,0.017461
# 0.023137,0.060074,0.034748,0.000366,0.000355,0.008674,0.225957,0.000360

# --> nice :)

In [ ]:
# Results[0][0]
# 2.164445,0.004278,0.921012,3.660252,1.532306,1.524151,4.093397,4.733766
# 3.453443,0.004395,0.898049,4.667126,1.998041,2.072289,11.121975,5.068848
# 2.233662,0.006844,0.710762,4.107797,1.644121,1.576521,14.268573,4.718986
# 0.385062,0.001257,0.100568,0.423951,0.210232,0.208753,16.626195,0.465520


# --> nice :)

In [ ]:
data[0][0][0][0]